In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
weights = models.ResNet18_Weights.DEFAULT
transform = weights.transforms()

In [ ]:

training_data = datasets.FGVCAircraft(
    root="data",
    split="train",
    download=True,
    transform=transform
)


test_data = datasets.FGVCAircraft(
    root="data",
    split="test",
    download=True,
    transform=transform
)

In [32]:
print(training_data)
print(test_data)

Dataset FGVCAircraft
    Number of datapoints: 3334
    Root location: data
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BILINEAR
           )
Dataset FGVCAircraft
    Number of datapoints: 3333
    Root location: data
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BILINEAR
           )


In [8]:
'''
1. Torch tensor
train_target = torch.tensor(train['Target'].values.astype(np.float32)) # train ist ein DF
train = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32))

2. Torch dataset
train_tensor = data_utils.TensorDataset(train, train_target)
test_tensor = data_utils.TensorDataset(test, test_target)

3. Torch dataloader
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)
'''


"\n1. Torch tensor\ntrain_target = torch.tensor(train['Target'].values.astype(np.float32)) # train ist ein DF\ntrain = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32))\n\n2. Torch dataset\ntrain_tensor = data_utils.TensorDataset(train, train_target)\ntest_tensor = data_utils.TensorDataset(test, test_target)\n\n3. Torch dataloader\ntrain_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)\ntest_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)\n"

In [ ]:
# Dataloaders will be created after hyperparameters are set

In [33]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model suitable for FGVCAircraft (100 classes, 224x224 RGB)
num_classes = len(training_data.classes)
# Use a pretrained ResNet18 and replace the final fully connected layer
resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
model = resnet.to(device)

# Hyperparameters
learning_rate = 1e-4
batch_size = 32
epochs = 2

Using cpu device
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/isc-den/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [34]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            current = (batch + 1) * len(X)
            print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

In [37]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    test_loss, correct = 0.0, 0.0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss.item() * X.size(0)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {accuracy*100:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 2
# Create dataloaders
loader_kwargs = {"batch_size": batch_size, "shuffle": True}
if device == "cuda":
    loader_kwargs.update({"num_workers": 2, "pin_memory": True})
train_dataloader = DataLoader(training_data, **loader_kwargs)
loader_kwargs["shuffle"] = False
test_dataloader = DataLoader(test_data, **loader_kwargs)

# Quick sanity check
X0, y0 = next(iter(train_dataloader))
print("Batch shape:", X0.shape, "Labels shape:", y0.shape)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Batch shape: torch.Size([32, 3, 224, 224]) Labels shape: torch.Size([32])
Epoch 1
-------------------------------
loss: 1.796173  [   32/ 3334]
loss: 1.465021  [ 1632/ 3334]
loss: 1.366227  [ 3232/ 3334]
Test Error: 
 Accuracy: 53.0%, Avg loss: 2.146276 

Epoch 2
-------------------------------
loss: 0.798728  [   32/ 3334]
loss: 0.518741  [ 1632/ 3334]
loss: 0.469396  [ 3232/ 3334]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.981782 

Done!


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [40]:
# Reload model for inference
inference_model = models.resnet18(weights=None)
inference_model.fc = nn.Linear(inference_model.fc.in_features, num_classes)
inference_model = inference_model.to(device)
inference_model.load_state_dict(torch.load("model.pth", map_location=device))
inference_model.eval()

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer2.0.conv1.weight", "layer2.0.bn1.weight", "layer2.0.bn1.bias", "layer2.0.bn1.running_mean", "layer2.0.bn1.running_var", "layer2.0.conv2.weight", "layer2.0.bn2.weight", "layer2.0.bn2.bias", "layer2.0.bn2.running_mean", "layer2.0.bn2.running_var", "layer2.0.downsample.0.weight", "layer2.0.downsample.1.weight", "layer2.0.d

In [39]:
# Use dataset's class names
classes = training_data.classes

# Single-sample inference
x, y = test_data[0]
with torch.no_grad():
    x = x.unsqueeze(0).to(device)  # add batch dim
    logits = inference_model(x)
    pred_idx = int(logits.argmax(1).item())
    predicted, actual = classes[pred_idx], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

NameError: name 'inference_model' is not defined

In [ ]:
print("Number of test samples:", len(test_data))
print("Sample tensor shape:", test_data[0][0].shape)